## **Model Performance Results**

**Utils**

In [1]:
import json
def read_json(file_path): 
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [2]:
import pandas as pd
import numpy as np

def extract_metrics(results_dict, positive_label="1"):
    cm = results_dict["confusion_matrix"]
    tn, fp = cm[0]
    fn, tp = cm[1]

    # Recall for each class
    recall_pos = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    recall_neg = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    gmean = np.sqrt(recall_pos * recall_neg)

    return {
        "Accuracy": results_dict["accuracy"],
        "Balanced Accuracy": results_dict["balanced_accuracy"],
        "Precision (Crisis)": results_dict["classification_report"][positive_label]["precision"],
        "Recall (Crisis)": recall_pos,
        "F1-score (Crisis)": results_dict["classification_report"][positive_label]["f1-score"],
        "G-Mean": gmean
    }


#### **Models trained with Baseline Dataset**

In [3]:
results_base = read_json('../outputs/results_base.json')
results_oversampling = read_json('../outputs/results_oversampling.json')
results_focal_loss = read_json('../outputs/results_focal_loss.json')
results_class_weights = read_json('../outputs/results_class_weights.json')
results_cost_sensitive = read_json('../outputs/results_cost_sensitive.json')

In [4]:
results_table = pd.DataFrame.from_dict(
    {
        "Baseline LSTM": extract_metrics(results_base),
        "Window Oversampling": extract_metrics(results_oversampling),
        "Focal Loss": extract_metrics(results_focal_loss),
        "Class Weights": extract_metrics(results_class_weights),
        "Adaptive Cost-Sensitive": extract_metrics(results_cost_sensitive),
    },
    orient="index"
)


results_table = results_table.round(3)
results_table

,Accuracy,Balanced Accuracy,Precision (Crisis),Recall (Crisis),F1-score (Crisis),G-Mean
Baseline LSTM,0.200,0.565,0.091,1.0,0.167,0.361
Window Oversampling,0.640,0.804,0.182,1.0,0.308,0.780
Focal Loss,0.440,0.696,0.125,1.0,0.222,0.626
Class Weights,0.762,0.875,0.167,1.0,0.286,0.866
Adaptive Cost-Sensitive,0.714,0.850,0.143,1.0,0.250,0.837


In [5]:
print(
    results_table.to_latex(
        float_format="%.3f",
        caption="Performance comparison of imbalance-handling techniques",
        label="tab:imbalance_results"
    )
)

\begin{table}
\caption{Performance comparison of imbalance-handling techniques}
\label{tab:imbalance_results}
\begin{tabular}{lrrrrrr}
\toprule
 & Accuracy & Balanced Accuracy & Precision (Crisis) & Recall (Crisis) & F1-score (Crisis) & G-Mean \\
\midrule
Baseline LSTM & 0.200 & 0.565 & 0.091 & 1.000 & 0.167 & 0.361 \\
Window Oversampling & 0.640 & 0.804 & 0.182 & 1.000 & 0.308 & 0.780 \\
Focal Loss & 0.440 & 0.696 & 0.125 & 1.000 & 0.222 & 0.626 \\
Class Weights & 0.762 & 0.875 & 0.167 & 1.000 & 0.286 & 0.866 \\
Adaptive Cost-Sensitive & 0.714 & 0.850 & 0.143 & 1.000 & 0.250 & 0.837 \\
\bottomrule
\end{tabular}
\end{table}



### **Models trained with Imputed Dataset**

In [6]:
results_base = read_json('../outputs/results_base_imputed.json')
results_oversampling = read_json('../outputs/results_oversampling_imputed.json')
results_focal_loss = read_json('../outputs/results_focal_loss_imputed.json')
results_class_weights = read_json('../outputs/results_class_weights_imputed.json')
results_cost_sensitive = read_json('../outputs/results_cost_sensitive_imputed.json')

In [7]:
results_table_imputed = pd.DataFrame.from_dict(
    {
        "Baseline Imputed LSTM": extract_metrics(results_base),
        "Window Oversampling Imputed": extract_metrics(results_oversampling),
        "Focal Loss Imputed": extract_metrics(results_focal_loss),
        "Class Weights Imputed": extract_metrics(results_class_weights),
        "Adaptive Cost-Sensitive Imputed": extract_metrics(results_cost_sensitive),
    },
    orient="index"
)

results_table_imputed = results_table_imputed.round(3)
results_table_imputed

,Accuracy,Balanced Accuracy,Precision (Crisis),Recall (Crisis),F1-score (Crisis),G-Mean
Baseline Imputed LSTM,0.465,0.720,0.080,1.0,0.148,0.663
Window Oversampling Imputed,0.628,0.805,0.111,1.0,0.200,0.781
Focal Loss Imputed,0.744,0.866,0.154,1.0,0.267,0.855
Class Weights Imputed,0.744,0.865,0.167,1.0,0.286,0.854
Adaptive Cost-Sensitive Imputed,0.795,0.655,0.125,0.5,0.200,0.637


----

### **20-runs Experiment**

In [8]:
def extract_metrics_aggregated(results_dict):
    return {
        "Accuracy": f"{results_dict['accuracy']['mean']:.3f} ± {results_dict['accuracy']['std']:.3f}",
        "Balanced Accuracy": f"{results_dict['balanced_accuracy']['mean']:.3f} ± {results_dict['balanced_accuracy']['std']:.3f}",
        "Precision (Crisis)": f"{results_dict['precision_crisis']['mean']:.3f} ± {results_dict['precision_crisis']['std']:.3f}",
        "Recall (Crisis)": f"{results_dict['recall_crisis']['mean']:.3f} ± {results_dict['recall_crisis']['std']:.3f}",
        "F1-score (Crisis)": f"{results_dict['f1_crisis']['mean']:.3f} ± {results_dict['f1_crisis']['std']:.3f}",
        "G-Mean": f"{results_dict['g_mean']['mean']:.3f} ± {results_dict['g_mean']['std']:.3f}",
    }


1. #### **Models trained with Baseline Dataset**

In [11]:
results_base = read_json("../outputs/base_lstm_aggregated.json")
results_oversampling = read_json("../outputs/oversampling_aggregated.json")
results_focal = read_json("../outputs/focal_loss_aggregated.json")
results_class_weights = read_json("../outputs/class_weights_aggregated.json")
results_cost_sensitive = read_json("../outputs/cost_sensitive_aggregated.json")

results_table = pd.DataFrame.from_dict(
    {
        "Baseline LSTM": extract_metrics_aggregated(results_base),
        "Window Oversampling": extract_metrics_aggregated(results_oversampling),
        "Focal Loss": extract_metrics_aggregated(results_focal),
        "Class Weights": extract_metrics_aggregated(results_class_weights),
        "Adaptive Cost-Sensitive": extract_metrics_aggregated(results_cost_sensitive),
    },
    orient="index"
)

results_table

,Accuracy,Balanced Accuracy,Precision (Crisis),Recall (Crisis),F1-score (Crisis),G-Mean
Baseline LSTM,0.248 ± 0.078,0.591 ± 0.042,0.097 ± 0.010,1.000 ± 0.000,0.177 ± 0.016,0.416 ± 0.099
Window Oversampling,0.624 ± 0.102,0.659 ± 0.096,0.140 ± 0.033,0.700 ± 0.245,0.230 ± 0.051,0.637 ± 0.086
Focal Loss,0.306 ± 0.111,0.623 ± 0.060,0.106 ± 0.016,1.000 ± 0.000,0.191 ± 0.026,0.472 ± 0.152
Class Weights,0.819 ± 0.171,0.857 ± 0.147,0.328 ± 0.316,0.900 ± 0.300,0.423 ± 0.304,0.796 ± 0.285
Adaptive Cost-Sensitive,0.595 ± 0.160,0.764 ± 0.106,0.107 ± 0.041,0.950 ± 0.218,0.191 ± 0.069,0.703 ± 0.195


In [13]:
print(
    results_table.to_latex(
        float_format="%.3f",
        caption="Performance comparison of imbalance-handling techniques using the Baseline Dataset",
        label="tab:imbalance_results"
    )
)

\begin{table}
\caption{Performance comparison of imbalance-handling techniques using the Baseline Dataset}
\label{tab:imbalance_results}
\begin{tabular}{lllllll}
\toprule
 & Accuracy & Balanced Accuracy & Precision (Crisis) & Recall (Crisis) & F1-score (Crisis) & G-Mean \\
\midrule
Baseline LSTM & 0.248 ± 0.078 & 0.591 ± 0.042 & 0.097 ± 0.010 & 1.000 ± 0.000 & 0.177 ± 0.016 & 0.416 ± 0.099 \\
Window Oversampling & 0.624 ± 0.102 & 0.659 ± 0.096 & 0.140 ± 0.033 & 0.700 ± 0.245 & 0.230 ± 0.051 & 0.637 ± 0.086 \\
Focal Loss & 0.306 ± 0.111 & 0.623 ± 0.060 & 0.106 ± 0.016 & 1.000 ± 0.000 & 0.191 ± 0.026 & 0.472 ± 0.152 \\
Class Weights & 0.819 ± 0.171 & 0.857 ± 0.147 & 0.328 ± 0.316 & 0.900 ± 0.300 & 0.423 ± 0.304 & 0.796 ± 0.285 \\
Adaptive Cost-Sensitive & 0.595 ± 0.160 & 0.764 ± 0.106 & 0.107 ± 0.041 & 0.950 ± 0.218 & 0.191 ± 0.069 & 0.703 ± 0.195 \\
\bottomrule
\end{tabular}
\end{table}



2. #### **Models trained with Imputed Dataset**

In [10]:
results_base = read_json("../outputs/base_imputed_lstm_aggregated.json")
results_oversampling = read_json("../outputs/oversampling_imputed_aggregated.json")
results_focal = read_json("../outputs/focal_loss_imputed_aggregated.json")
results_class_weights = read_json("../outputs/class_weights_imputed_aggregated.json")
results_cost_sensitive = read_json("../outputs/cost_sensitive_imputed_aggregated.json")

results_table_imputed = pd.DataFrame.from_dict(
    {
        "Baseline LSTM": extract_metrics_aggregated(results_base),
        "Window Oversampling": extract_metrics_aggregated(results_oversampling),
        "Focal Loss": extract_metrics_aggregated(results_focal),
        "Class Weights": extract_metrics_aggregated(results_class_weights),
        "Adaptive Cost-Sensitive": extract_metrics_aggregated(results_cost_sensitive),
    },
    orient="index"
)

results_table_imputed

,Accuracy,Balanced Accuracy,Precision (Crisis),Recall (Crisis),F1-score (Crisis),G-Mean
Baseline LSTM,0.486 ± 0.062,0.563 ± 0.032,0.103 ± 0.014,1.000 ± 0.000,0.186 ± 0.022,0.541 ± 0.043
Window Oversampling,0.624 ± 0.032,0.655 ± 0.089,0.101 ± 0.018,0.900 ± 0.200,0.181 ± 0.033,0.634 ± 0.082
Focal Loss,0.586 ± 0.058,0.771 ± 0.053,0.100 ± 0.015,0.975 ± 0.109,0.181 ± 0.025,0.740 ± 0.055
Class Weights,0.622 ± 0.100,0.802 ± 0.085,0.117 ± 0.029,0.925 ± 0.179,0.206 ± 0.048,0.768 ± 0.090
Adaptive Cost-Sensitive,0.621 ± 0.119,0.765 ± 0.108,0.120 ± 0.038,0.925 ± 0.179,0.211 ± 0.062,0.739 ± 0.113


In [14]:
print(
    results_table.to_latex(
        float_format="%.3f",
        caption="Performance comparison of imbalance-handling techniques using the Imputed Dataset",
        label="tab:imbalance_results"
    )
)

\begin{table}
\caption{Performance comparison of imbalance-handling techniques using the Imputed Dataset}
\label{tab:imbalance_results}
\begin{tabular}{lllllll}
\toprule
 & Accuracy & Balanced Accuracy & Precision (Crisis) & Recall (Crisis) & F1-score (Crisis) & G-Mean \\
\midrule
Baseline LSTM & 0.248 ± 0.078 & 0.591 ± 0.042 & 0.097 ± 0.010 & 1.000 ± 0.000 & 0.177 ± 0.016 & 0.416 ± 0.099 \\
Window Oversampling & 0.624 ± 0.102 & 0.659 ± 0.096 & 0.140 ± 0.033 & 0.700 ± 0.245 & 0.230 ± 0.051 & 0.637 ± 0.086 \\
Focal Loss & 0.306 ± 0.111 & 0.623 ± 0.060 & 0.106 ± 0.016 & 1.000 ± 0.000 & 0.191 ± 0.026 & 0.472 ± 0.152 \\
Class Weights & 0.819 ± 0.171 & 0.857 ± 0.147 & 0.328 ± 0.316 & 0.900 ± 0.300 & 0.423 ± 0.304 & 0.796 ± 0.285 \\
Adaptive Cost-Sensitive & 0.595 ± 0.160 & 0.764 ± 0.106 & 0.107 ± 0.041 & 0.950 ± 0.218 & 0.191 ± 0.069 & 0.703 ± 0.195 \\
\bottomrule
\end{tabular}
\end{table}

